In [1]:
import msgpack

with open('SQuAD/data.msgpack', 'rb') as f:
    data = msgpack.load(f, encoding='utf8')
# train = sorted(data['train'], key=lambda x: len(x[1]))
# dev = data['dev'].sort(key=lambda x: len(x[1]))


In [28]:
# train: id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer_start, answer_end
# dev:   id, context_id, context_features, tag_id, ent_id,
#        question_id, context, context_token_span, answer
import torch
import re
import traceback
def train_data_trans(row):
    id, context_id, context_features, tag_id, ent_id,\
    question_id, context, context_token_span, answer_start, answer_end = row
    return len(context_id), len(question_id), id, torch.tensor(context_id), torch.tensor(question_id), \
           [torch.tensor(context_id[answer_start:answer_end + 1])], [torch.tensor([answer_start, answer_end])]
def train_data_dev(row):
    id, context_id, context_features, tag_id, ent_id,\
    question_id, context, context_token_span, answers = row
    span_list = []
    answer_list = []
    try:
        for answer in answers:
            answer_match = re.search(answer, context)
            if answer_match is not None:
                answer_start = list(zip(*context_token_span))[0].index(answer_match.start())
                answer_end = list(zip(*context_token_span))[1].index(answer_match.end())
                answer_list.append(torch.tensor(context_id[answer_start:answer_end + 1]))
                span_list.append(torch.tensor([answer_start, answer_end]))
        return len(context_id), len(question_id), id, torch.tensor(context_id), torch.tensor(question_id), \
               answer_list, span_list
    except:
        print(id)
        print(answer_match)
        print(context_token_span)
        traceback.print_exc()

In [2]:
from pre import flatten_json
train_json = flatten_json('.data/train-v1.1.json', 'train')
dev_json = flatten_json('.data/dev-v1.1.json', 'dev')

In [15]:
dev_0 = data['dev'][1]
# t_json_0 = train_json[0]
trans_0 = train_data_dev(dev_0)

In [19]:
import pickle
trans_train = [train_data_trans(row) for row in data['train']].sort()
trans_dev = [train_data_dev(row) for row in data['dev']].sort()
with open('trans_train.pickle', 'wb') as f:
    pickle.dump(trans_train, f, pickle.HIGHEST_PROTOCOL)
with open('trans_dev.pickle', 'wb') as f:
    pickle.dump(trans_dev, f, pickle.HIGHEST_PROTOCOL)

56e12005cd28a01900c67619
56e12005cd28a01900c6761a
56d722330d65d214001983a6
56d722330d65d214001983a8
56d9be16dc89441400fdb770
56f802c6a6d7ea1400e1737b
56bf738b3aeaaa14008c9655
56e11c24e3433e1400422c1a
57060a1175f01819005e78d3
56e1f10ee3433e1400423223
56be4eafacb8001400a50303
56bf17653aeaaa14008c9514
56bf17653aeaaa14008c9515
56d204ade7d4791d00902604
56bec2013aeaaa14008c9373
56e0ef90231d4119001ac46b
56dfb0c8231d4119001abc85
56e0d54a7aa994140058e76d
56bec0353aeaaa14008c9350
56bec0353aeaaa14008c9351
56bf5b933aeaaa14008c95f1
56d71d9d0d65d21400198377
570602fa52bb8914006897a0
56e16182e3433e1400422e2a
5705e63175f01819005e7724
56dfa24a38dc421700152142
56dfa24a38dc421700152146
56e0c1617aa994140058e6d7
56e0c1617aa994140058e6d9
5725f39638643c19005acef8
57263cfcec44d21400f3dc8d
5727515f708984140094dc14
572975511d046914007794a7
5733a2a9d058e614000b5f2b
572914f46aef051400154a46
5729582b1d046914007792e3
57376df3c3c5551400e51ed7
56f8c9719e9bad19000a04e1
57108198b654c5140001f93b
57273887dd62a815002e99a4


TypeError: '<' not supported between instances of 'NoneType' and 'tuple'

In [29]:
dev_wrong = [item for item in data['dev']
             if item[0] == '56e12005cd28a01900c67619'][0]
train_data_dev(dev_wrong)

56e12005cd28a01900c67619
<re.Match object; span=(127, 137), match='her weight'>
[[0, 5], [6, 11], [12, 14], [15, 20], [21, 23], [24, 29], [30, 33], [34, 40], [41, 50], [51, 58], [59, 62], [63, 73], [74, 80], [80, 81], [82, 86], [87, 89], [90, 94], [95, 97], [98, 103], [104, 105], [106, 115], [116, 123], [124, 126], [127, 130], [131, 142], [143, 145], [146, 149], [150, 155], [156, 158], [159, 168], [169, 177], [177, 178], [179, 181], [182, 189], [190, 199], [199, 200], [201, 206], [207, 215], [216, 217], [218, 229], [230, 232], [233, 235], [236, 240], [241, 244], [245, 251], [252, 255], [256, 265]]


Traceback (most recent call last):
  File "<ipython-input-28-9109ba18905e>", line 23, in train_data_dev
    answer_end = list(zip(*context_token_span))[1].index(answer_match.end())
ValueError: tuple.index(x): x not in tuple


In [31]:
dev_wrong[6]

'Tesla could be harsh at times and openly expressed disgust for overweight people, such as when he fired a secretary because of her weight.:110 He was quick to criticize clothing; on several occasions, Tesla directed a subordinate to go home and change her dress.:33'